In [134]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras.activations import softmax
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *
from keras.models import model_from_json
import json



In [135]:
def sanitize(string):
    words = string.split(' ')
    return words

In [136]:
!nvidia-smi


Tue Jul 16 01:38:41 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 27%   30C    P8    20W / 250W |  10827MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
#read csv

train = pd.read_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv", sep='\t')
dev = pd.read_csv("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv", sep='\t')


print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sent1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sent2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sent1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sent2'] \
         .apply(sanitize)


In [13]:
train.head()

,Unnamed: 0,TRAIN_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label,title1_tokenized,title2_tokenized
0,0,4,44326,b'What are the most common traffic convictions...,b'What are the most common traffic convictions...,most common,common,O O O B E O O O O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O O O O,0,"[b'What, are, the, most, common, traffic, conv...","[b'What, are, the, most, common, traffic, conv..."
1,1,10,103288,b'Which are in your opinion the most beautiful...,b'Which are in your opinion the most depressin...,most beautiful,beautiful,O O O O O O B E O O O O O O O O,O O O O O O O O O O O S O O O O,1,"[b'Which, are, in, your, opinion, the, most, b...","[b'Which, are, in, your, opinion, the, most, d..."
2,2,14,316010,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ...",nuclear energy,nuclear,O B E O O O,O O S O O O O O O O,1,"[b'Is, nuclear, energy, renewable, energy, ?']","[b""``, Is, nuclear, energy, ``, renewable, ene..."
3,3,18,526060,b'What ancient units would definitely win in a...,b'What numerous modern units would definitely ...,fight against,fight,O O O O O O O O O B E O O O O,O O O O O O O O O O S O O O O,0,"[b'What, ancient, units, would, definitely, wi...","[b'What, numerous, modern, units, would, defin..."
4,4,61,43995,b'I have only a very nice startup idea but I m...,b'I have only a very good startup idea but I m...,very nice,nice,O O O O B E O O O O O O O O O O O O O,O O O O O O O O O O O O S O O O O O O,1,"[b'I, have, only, a, very, nice, startup, idea...","[b'I, have, only, a, very, good, startup, idea..."


In [108]:
embeddings_index = {}
f = open('GloVEs/glove.840B.300d.txt')
for line in f:
    values = line.split(' ')
    word = values[0] ## The first entry is the word
    coefs = np.asarray(values[1:], dtype='float32') ## These are the vecotrs representing the embedding for the word
    embeddings_index[word] = coefs

f.close()

print('GloVe data loaded')

KeyboardInterrupt: 

In [23]:
train_Tb.column("sent1")

array(["b'What are the most common traffic convictions in Arizona , and how does the severity of the convictions differ in Arkansas ?'",
       "b'Which are in your opinion the most beautiful and the most depressing truths about life ?'",
       "b'Is nuclear energy renewable energy ?'", ...,
       "b'Did Barnacle really raise $ 7M just to close a few months later ?'",
       'b"What are the major flaws with Donald Trump \'s and Hillary Clinton \'s foreign policy agendas ?"',
       'b"`` Why do many Europeans think Americans are `` dumb `` even when the U.S. has the highest GDP in the world ? \'\'"'],
      dtype='<U244')

In [114]:
#tokenize sentences
def normalize_and_split(sentence):
    s=sentence.split()
    l=[x.lower() for x in s]
    first_word=(l[0])[2:]
    l[0]=first_word
    return l


#tokenize labels

def t_label(str):
    ret=[]
    s = str.split() #split into individual alphabets
    for x in s:
        if x=='O':
            ret.append(0)
        else:
            ret.append(1)
    for t in range(60-len(s)): 
        ret.append(0)
    return ret


60

In [65]:
normalize_and_split("b'I Am LiKINg iTt")

['i', 'am', 'liking', 'itt']

In [89]:
train_Tb=Table.read_table("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_TRAIN.tsv")
dev_Tb=Table.read_table("dataset/Phrase_Level/PAWS_QQP/LABELED_PAWS_QQP_DEV.tsv")

In [90]:
#if maintain only one extraction per pair; drop those repeats

to_drop=[]
to_drop2=[]
id = train_Tb.column(1)
id2=dev_Tb.column(1)


for x in range(train_Tb.num_rows-1):
    if id[x]==id[x+1]:
        to_drop.append(x)
    else:
        continue
        
        
for x in range(dev_Tb.num_rows-1):
    if id2[x]==id2[x+1]:
        to_drop2.append(x)
    else:
        continue


train_Tb=train_Tb.exclude(to_drop)             
train_Tb.show(20)

dev_Tb=dev_Tb.exclude(to_drop2)             
dev_Tb.show(20)

Unnamed: 0,TRAIN_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label
0,4,44326,b'What are the most common traffic convictions in Arizon ...,b'What are the most common traffic convictions in Arkans ...,most common,common,O O O B E O O O O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O O O O,0
1,10,103288,b'Which are in your opinion the most beautiful and the m ...,b'Which are in your opinion the most depressing and the ...,most beautiful,beautiful,O O O O O O B E O O O O O O O O,O O O O O O O O O O O S O O O O,1
2,14,316010,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",nuclear energy,nuclear,O B E O O O,O O S O O O O O O O,1
3,18,526060,b'What ancient units would definitely win in a 1v1 fight ...,b'What numerous modern units would definitely win in a 1 ...,fight against,fight,O O O O O O O O O B E O O O O,O O O O O O O O O O S O O O O,0
5,61,464375,b'I have only a very nice startup idea but I m not good ...,b'I have only a very good startup idea but I m not nice ...,very nice,nice,O O O O B E O O O O O O O O O O O O O,O O O O O O O O O O O O S O O O O O O,1
6,65,566357,"b'Do women feel more heat than men , given that they wea ...","b'Do women feel less heat than men , given that they wea ...",surface area,surface,O O O O O O O O O O O O O O O O O B E O O O O,O O O O O O O O O O O O O O O O O S O O O O O,0
7,82,321222,b'Why does volumetric strain volume of a solid body alwa ...,b'Why does volumetric strain volume of a solid body alwa ...,give an example,example,O O O O O O O O O O O O O O O O O O O B I E O,O O O O O O O O O O O O O O O O O O O O O S O,0
8,119,44326,b'What are the most common fee structures charged for PP ...,b'What are the most common fee structures used for PPC M ...,most common,common,O O O B E O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O,0
9,133,560981,"b""I just turned 13 and I was about 5 '4 . How much shoul ...","b""I just turned 13 and I 'm around 5 '4 . How much shoul ...",around 150,about 150,O O O O O O O O O O O O O O O O O O O B E O O O O O O,O O O O O O O O O O O O O O O O O O O B E O O O O O O,1
11,143,184768,"b""`` Is 1 month too early to say `` I love you `` ? What ...",b'Is 1 month too early to say I love you ? What is the n ...,too early,early,O O O O B E O O O O O O O O O O O O O O O O O O,O O O O S O O O O O O O O O O O O O O O,1


Unnamed: 0,DEV_PAWS_QQP_id,PPDB_id,sent1,sent2,paraphrase1,paraphrase2,labels1,labels2,Ground Label
1,3,129607,"b""Why ca n't countries afford high quality products , bu ...","b""Why ca n't countries afford China 's high quality prod ...",high quality,quality,O O O O O B E O O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O O,0
3,11,366448,b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...,begin with,start with,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,O O O O O O O O O O O B E O O O O O O O O O O O O O O O ...,1
4,22,543619,"b""I 'm currently working for a Big 4 as an Advanced Tax ...","b""I 'm currently working for a Big 4 as an Advanced Tax ...",much better,much,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,0
5,25,487239,"b'In Nevada when pulling out of a parking lot , can you ...","b'In Nevada when pulling out of a parking lot , can you ...",far left,left,O O O O O O O O O O O O O O O O O O B E O O O O O O O O ...,O O O O O O O O O O O O O O S O O O O O O O O O O O O O ...,0
6,29,284980,"b""What are the reasons for SAARC 's failure as regional ...","b""What are the reasons for SAARC 's failure as common or ...",economic cooperation,cooperation,O O O O O O O O O O O O O O O B E O O O O O O O,O O O O O O O O O O O O O O O O O O O O O O S O,0
7,56,459668,"b'I want to be a child psychologist , what qualification ...","b'I want to become a child psychologist , what qualifica ...",become one,be,O O O O O O O O O O O O O O B E O O O O O O O O O O O O O,O O O O O O O O O O O O O O S O O O O O O O O O O O O O O,1
9,57,494710,b'Why are beautiful girls so rude all the time ? I saw a ...,b'Why are rude girls so beautiful all the time ? I saw a ...,go away,away,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,O O O O O O O O O O O O O O O O O O O O O O O O O O O O ...,0
11,61,65644,"b""What is the name of the creepiest horror movie you 've ...","b""What is the name of the scariest horror movie you 've ...",ever seen,'ve ever seen,O O O O O O O O O O O B E O O O O O O O,O O O O O O O O O O B I E O O O O O O O,0
15,62,408259,b'I am an electrical engineering student.I am very much ...,"b'Am an electrical engineering student , am very much in ...",very much,much,O O O O O O O B E O O O O O O O O O O O O O O O O O O,O O O O O O O O S O O O O O O O O O O O O O O O O O O,0
16,70,573133,"b""What is the purpose of life ? What 's life actually ab ...","b""What 's the purpose of life ? What is life actually ab ...",'s life,life,O O O O O O O O B E O O O,O O O O O S O O O O O O O,1


In [115]:
#normalize and tokenize sentences

train_Tb_sen1=train_Tb.apply(normalize_and_split, "sent1")
train_Tb_sen2=train_Tb.apply(normalize_and_split, "sent2")
train_Tb_labels1=train_Tb.apply(t_label, "labels1")
train_Tb_labels2=train_Tb.apply(t_label, "labels2")

dev_Tb_sen1=dev_Tb.apply(normalize_and_split, "sent1")
dev_Tb_sen2=dev_Tb.apply(normalize_and_split, "sent2")
dev_Tb_labels1=dev_Tb.apply(t_label, "labels1")
dev_Tb_labels2=dev_Tb.apply(t_label, "labels2")


In [116]:
trainTb=train_Tb.with_columns("sent1_token", train_Tb_sen1, "sent2_token", train_Tb_sen2, "label1_token", train_Tb_labels1, "label2_token", train_Tb_labels2)
devTb=dev_Tb.with_columns("sent1_token", dev_Tb_sen1, "sent2_token", dev_Tb_sen2,"label1_token", dev_Tb_labels1, "label2_token", dev_Tb_labels2)

In [100]:
max=0
for x in trainTb.column("sent1_token"):
    if max<len(x):
        max=len(x)
print(max)

max=0
for x in trainTb.column("sent2_token"):
    if max<len(x):
        max=len(x)
print(max)

max=0
for x in devTb.column("sent1_token"):
    if max<len(x):
        max=len(x)
print(max)

max=0
for x in devTb.column("sent2_token"):
    if max<len(x):
        max=len(x)
print(max)

#max seq length say 60

48
48
57
58


In [127]:
tl1=trainTb.column("label1_token")
tl2=trainTb.column("label2_token")
dl1=devTb.column("label1_token")
dl2=devTb.column("label2_token")

In [106]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


MAX_NUM_WORDS = 1000
MAX_SEQUENCE_LENGTH = 60

train_corpus = np.append(trainTb.column("sent1_token"), trainTb.column("sent2_token"))

#create a dictionary

tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)

x1_train = tokenizer \
    .texts_to_sequences(train_Tb_sen1)
x2_train = tokenizer \
    .texts_to_sequences(train_Tb_sen2)


print(tokenizer.document_count)



dev_corpus = np.append(devTb.column("sent1_token"), devTb.column("sent2_token"))
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_Tb_sen1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_Tb_sen2)

print(tokenizer.document_count)



word_index = tokenizer.word_index





1582
1708


In [124]:

x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)






In [126]:
x1_train[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  10,
        12,   2,  28, 116, 166,  91,   6, 562,   7,   5,  18,  35,   2,
       167,  15,   2,  91, 136,   6, 160,   1], dtype=int32)

In [109]:
## More code adapted from the keras reference (https://github.com/keras-team/keras/blob/master/examples/pretrained_word_embeddings.py)
# prepare embedding matrix 
from keras.layers import Embedding
from keras.initializers import Constant

## EMBEDDING_DIM =  ## seems to need to match the embeddings_index dimension
EMBEDDING_DIM = 300
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word) ## This references the loaded embeddings dictionary
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [110]:
class HAttLayer(Layer):
    def __init__(self, attention_dim):
        self.init = initializers.get('normal')
        self.supports_masking = True
        self.attention_dim = attention_dim
        super(HAttLayer, self).__init__()

    def build(self, input_shape):
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)))
        self.b = K.variable(self.init((self.attention_dim, )))
        self.u = K.variable(self.init((self.attention_dim, 1)))
        self.trainable_weights = [self.W, self.b, self.u]
        super(HAttLayer, self).build(input_shape)

    def compute_mask(self, inputs, mask=None):
        return mask

    def call(self, x, mask=None):
        # size of x :[batch_size, sel_len, attention_dim]
        # size of u :[batch_size, attention_dim]
        # uit = tanh(xW+b)
        uit = K.tanh(K.bias_add(K.dot(x, self.W), self.b))
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)

        ait = K.exp(ait)

        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            ait *= K.cast(mask, K.floatx())
        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = x * ait
        output = K.sum(weighted_input, axis=1)

        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])



In [120]:
#BiLSTM with glove


#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = EMBEDDING_DIM

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)

BiLSTM1 = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True))

BiLSTM2 = Bidirectional(LSTM(NUM_LSTM_UNITS, return_sequences=True))

top_output = BiLSTM1(top_embedded)

bm_output = BiLSTM2(bm_embedded)


modelHA_top = HAttLayer(attention_dim=NUM_EMBEDDING_DIM)(top_output)


modelHA_bm = HAttLayer(attention_dim=NUM_EMBEDDING_DIM)(bm_output)         #Attention Layer


a = dot([modelHA_top, top_output], axes=-1)


b = dot([modelHA_bm, bm_output], axes=-1)

dense =  Dense(
    units=60, 
    activation='sigmoid')

predictions1 = dense(a)

predictions2 = dense(b)


model = Model(
    inputs=[top_input, bm_input], 
    outputs=[predictions1, predictions2])


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 30

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min')
mcp_save = ModelCheckpoint('Phrase_Models_H5/PAWS_QQP_PHRASE', save_best_only=True,  verbose=1, monitor='val_loss', mode='min')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 300)      300300      input_7[0][0]                    
                                                                 input_8[0][0]                    
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 60, 256)      439296      embedding_2[4][0]                
__________

In [133]:
!nvidia-smi

Tue Jul 16 00:43:28 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  On   | 00000000:D8:00.0 Off |                  N/A |
| 27%   30C    P8    20W / 250W |  10827MiB / 10989MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [132]:
history = model.fit(x=[x1_train, x2_train], y=[tl1, tl2], batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[mcp_save], validation_data=([x1_dev, x2_dev], [dl1, dl2]),
    
    shuffle=True
)

Train on 791 samples, validate on 63 samples
Epoch 1/30


ResourceExhaustedError: OOM when allocating tensor with shape[30720,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node h_att_layer_4/MatMul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node metrics_1/acc/Mean}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [47]:

model.save("BiLSTM_PAWS_QQP.h5")        ##first attempt
print("Saved model to disk")  

Saved model to disk


In [12]:
model = load_model("BiLSTM_PAWS_QQP.h5")


In [13]:

predicts = model.predict(
    [x1_train, x2_train])

In [14]:
print(len(predicts))

11863


In [15]:
real1=train.sentence1
real2=train.sentence2
realCat=train.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(len(predicts)):
    if (np.argmax(predicts[x])!=(train.label[x])):
        no=np.append(no, x)
        err1=np.append(err1, real1[x])
        err2=np.append(err2, real2[x])
        wrongCat = np.append(wrongCat, realCat[x])
print(1-(len(wrongCat)/len(predicts)))

0.7814212256596139


In [16]:
error_analysis = Table()


error_analysis = error_analysis.with_column("id", no, "sent1", err1, "sent2", err2, "label", wrongCat)


#false POSITIVE: swap two words
#false negative: swap two specific nouns
error_analysis.show(30)

id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
15,b'What should I solve and what or how should I revise fo ...,b'What should I revise and what or how should I solve fo ...,0
20,b'How is the relative ratio of brain waves ( alpha/beta/ ...,b'How is the ratio of relative brain waves ( alpha/beta/ ...,0
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1


id,sent1,sent2,label
1,b'How can you treat ocd ? Is there any helpful suggestio ...,b'How can you treat OCD ? Is there any helpful suggestio ...,1
7,b'What are some beautiful lines to comment on Beautiful ...,b'What are some Beautiful lines to comment on beautiful ...,1
8,b'Can small dogs breed with large dogs ?',b'Can small dogs breed with large dogs ?',1
13,b'Are Indian IT services companies like Tech Mahindra/ W ...,b'Are Indian IT services companies like Wipro/TCS/Tech . ...,1
14,b'Is nuclear energy renewable energy ?',"b""`` Is nuclear energy `` renewable energy `` ? ''""",1
21,b'How can I make an android app using Python 3 ? Is ther ...,b'How can I develop an android app using Python 3 ? Is t ...,1
24,b'Headbreak : What can I do to prevent heart break night ...,b'Headbreak : What can I do to prevent heart break night ...,1
25,b'When is latent heat positive ?',b'When is latent heat positive ?',1
28,"b""What does the phrase 'Do you feel me ' mean ?""","b""What does the phrase `Do you feel me'mean ?""",1
29,b'What are some unexpected things first-time visitors to ...,"b'What are some unexpected things , first-time visitors ...",1
